# Cohort matching

In this notebook, we will add to our dataset the HY scale at baseline and 3Y follow up. Additionally, we will perform cohort matching on age, gender and baseline HY scale using `pymatch`

### Imports

In [39]:
import os, glob, json
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup as bs
from os import path

### Add HY (baseline, 3Y)

In [40]:
# Import data
baselineData = "../data/ppmi-data/HY_Baseline_Stage.csv"
volumesDataFile = "../data/volume-data/sanitizedVolumes.csv"
baselineDf = pd.read_csv(baselineData)
volumesDf = pd.read_csv(volumesDataFile)
baselineDf = baselineDf[["PATNO", "EVENT_ID","hy_on"]]

# Init H&Y list
baselineStageList = []
followUpStageList = []

# Fetch stages per patient
for subId in volumesDf["subjectId"].values:
    baselineStage = baselineDf.loc[(baselineDf['PATNO'] == subId) & (baselineDf['EVENT_ID'] == "BL")]["hy_on"]
    followUpStage = baselineDf.loc[(baselineDf['PATNO'] == subId) & (baselineDf['EVENT_ID'] == "V08")]["hy_on"]
    
    baselineStageList.append(baselineStage.values[0]) if not baselineStage.empty else baselineStageList.append(-1)
    followUpStageList.append(followUpStage.values[0]) if not followUpStage.empty else followUpStageList.append(-1)

# Append stages to DF
volumesDf["initialHY"] = baselineStageList
volumesDf["followUpHY"] = followUpStageList

print(f"Shape of data before removing missing data: {volumesDf.shape}")

# Remove missing data
volumesDf = volumesDf[volumesDf.followUpHY != -1]
volumesDf = volumesDf[volumesDf.initialHY != -1]
volumesDf = volumesDf.dropna(subset=['initialHY', 'followUpHY'])
volumesDf = volumesDf.drop("Unnamed: 0", 1)

print(f"Shape of data after removing missing data: {volumesDf.shape}")

# Create label group (0: stable | 1: progressive)
volumesDf["group"] = (volumesDf["initialHY"] < volumesDf["followUpHY"]).astype(int)
volumesDf = volumesDf.rename(columns={"Thalamus-Proper": "Thalamus", "Cerebellum-Cortex": "CerebellumCortex", "Cerebellum-White-Matter": "CerebellumWM", "3rd-Ventricle": "V3", "4th-Ventricle": "V4"})
volumesDf

Shape of data before removing missing data: (151, 16)
Shape of data after removing missing data: (125, 15)


/Users/mohanadarafe/miniconda3/envs/torch/lib/python3.7/site-packages/ipykernel_launcher.py:30: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only


,subjectId,Pallidum,Putamen,Caudate,Thalamus,CerebellumCortex,CerebellumWM,V3,V4,Pons,SCP,Midbrain,Insula,initialHY,followUpHY,group
0,4037,4261.3,10223.0,7586.8,17351.7,116856.4,38230.0,934.2,1330.4,19552.781704,307.724746,6989.344230,6476,1.0,2.0,1
1,3168,3776.7,8200.9,5738.2,13200.4,91395.5,31986.1,1089.4,1339.5,14452.019213,281.394581,5453.716726,7346,2.0,3.0,1
2,3131,4523.6,9383.2,8577.0,16020.2,118487.3,34742.2,1719.7,2169.2,21000.357958,348.730585,8004.224865,8146,2.0,2.0,0
3,4024,3444.1,8405.3,5940.0,12945.9,93723.6,22075.2,1587.0,1690.8,13148.503931,297.837617,5969.685193,6497,2.0,2.0,0
4,4001,4174.6,11058.9,7890.2,15731.7,126094.9,29284.0,1650.6,2085.8,16901.777054,262.838901,7330.256842,7613,2.0,2.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,3822,3916.3,8508.5,6534.8,13407.2,120662.0,34627.3,1605.5,1856.3,15993.876755,316.770996,6418.796616,6877,1.0,2.0,1
147,3372,4797.0,10114.8,9268.2,15093.5,112521.2,36136.9,3301.4,3167.2,17399.059361,318.438874,6918.687371,7682,1.0,2.0,1
148,3589,3067.4,7619.4,6386.1,13104.2,92495.3,29665.6,1319.4,1262.7,14052.641567,283.262188,5467.899590,5476,2.0,2.0,0
149,3586,3709.8,8082.5,5973.9,13831.9,109396.4,29020.2,1464.9,1306.0,16415.510277,342.644554,6638.502535,6423,1.0,2.0,1


### Remove Verio Scanners

In [31]:
def parse_metadata():
    '''
    Get DatFrame table consisting of two columns: subjectId and scannerType
    '''
    df = pd.DataFrame(columns=["subjectId", "scannerType"])
    for index, mdFilePath in enumerate(glob.glob("../data/metadata/*")):
        with open(mdFilePath, "r") as file:
            content = file.readlines()
            content = "".join(content)
            bs_content = bs(content, "lxml")
            subjectId = bs_content.find("subject").find("subjectidentifier").getText()
            scannerType = bs_content.find_all("protocol", attrs={'term':'Manufacturer'})[0].string
            model = bs_content.find_all("protocol", attrs={'term':'Mfg Model'})[0].string
            df = df.append({
                "subjectId": int(subjectId),
                "scannerType": f"{scannerType} {model}"
            }, ignore_index=True)
            df["subjectId"] = df["subjectId"].astype('int64')

    return df

# Get metadata and subject ID's
scannerDf = parse_metadata()
ids = volumesDf["subjectId"].values

# Remove all patients that are not in our dataset and that use a different scanner
scannerDf = scannerDf[scannerDf['subjectId'].isin(ids)]
# scannerDf = scannerDf[scannerDf["scannerType"]=="SIEMENS TrioTim"]

# List of IDs to remove from volumesDf
subjectsToRemoveIDs = scannerDf[scannerDf["scannerType"]!="SIEMENS TrioTim"]["subjectId"].values

# Removing subjects
for subId in subjectsToRemoveIDs:
    volumesDf = volumesDf.drop(volumesDf[volumesDf["subjectId"]==subId].index)
    
volumesDf

,subjectId,Pallidum,Putamen,Caudate,Thalamus,CerebellumCortex,CerebellumWM,V3,V4,Pons,SCP,Midbrain,Insula,initialHY,followUpHY,group
0,4037,4261.3,10223.0,7586.8,17351.7,116856.4,38230.0,934.2,1330.4,19552.781704,307.724746,6989.344230,6476,1.0,2.0,1
1,3168,3776.7,8200.9,5738.2,13200.4,91395.5,31986.1,1089.4,1339.5,14452.019213,281.394581,5453.716726,7346,2.0,3.0,1
2,3131,4523.6,9383.2,8577.0,16020.2,118487.3,34742.2,1719.7,2169.2,21000.357958,348.730585,8004.224865,8146,2.0,2.0,0
3,4024,3444.1,8405.3,5940.0,12945.9,93723.6,22075.2,1587.0,1690.8,13148.503931,297.837617,5969.685193,6497,2.0,2.0,0
4,4001,4174.6,11058.9,7890.2,15731.7,126094.9,29284.0,1650.6,2085.8,16901.777054,262.838901,7330.256842,7613,2.0,2.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,3822,3916.3,8508.5,6534.8,13407.2,120662.0,34627.3,1605.5,1856.3,15993.876755,316.770996,6418.796616,6877,1.0,2.0,1
147,3372,4797.0,10114.8,9268.2,15093.5,112521.2,36136.9,3301.4,3167.2,17399.059361,318.438874,6918.687371,7682,1.0,2.0,1
148,3589,3067.4,7619.4,6386.1,13104.2,92495.3,29665.6,1319.4,1262.7,14052.641567,283.262188,5467.899590,5476,2.0,2.0,0
149,3586,3709.8,8082.5,5973.9,13831.9,109396.4,29020.2,1464.9,1306.0,16415.510277,342.644554,6638.502535,6423,1.0,2.0,1


### Add gender, age

In [41]:
# Read data
baselineDf = pd.read_csv(baselineData)
baselineDf = baselineDf[["PATNO", "age", "gen"]]
baselineDf = baselineDf.drop_duplicates().rename(columns={"PATNO":"subjectId"})

# Merge data
volumesDf = pd.merge(baselineDf, volumesDf, on=["subjectId"], how="right")
volumesDf

,subjectId,age,gen,Pallidum,Putamen,Caudate,Thalamus,CerebellumCortex,CerebellumWM,V3,V4,Pons,SCP,Midbrain,Insula,initialHY,followUpHY,group
0,4037,52.831492,1,4261.3,10223.0,7586.8,17351.7,116856.4,38230.0,934.2,1330.4,19552.781704,307.724746,6989.344230,6476,1.0,2.0,1
1,3168,63.094798,2,3776.7,8200.9,5738.2,13200.4,91395.5,31986.1,1089.4,1339.5,14452.019213,281.394581,5453.716726,7346,2.0,3.0,1
2,3131,71.205479,1,4523.6,9383.2,8577.0,16020.2,118487.3,34742.2,1719.7,2169.2,21000.357958,348.730585,8004.224865,8146,2.0,2.0,0
3,4024,72.292350,1,3444.1,8405.3,5940.0,12945.9,93723.6,22075.2,1587.0,1690.8,13148.503931,297.837617,5969.685193,6497,2.0,2.0,0
4,4001,49.893151,1,4174.6,11058.9,7890.2,15731.7,126094.9,29284.0,1650.6,2085.8,16901.777054,262.838901,7330.256842,7613,2.0,2.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120,3822,55.994536,1,3916.3,8508.5,6534.8,13407.2,120662.0,34627.3,1605.5,1856.3,15993.876755,316.770996,6418.796616,6877,1.0,2.0,1
121,3372,71.390710,1,4797.0,10114.8,9268.2,15093.5,112521.2,36136.9,3301.4,3167.2,17399.059361,318.438874,6918.687371,7682,1.0,2.0,1
122,3589,74.909589,2,3067.4,7619.4,6386.1,13104.2,92495.3,29665.6,1319.4,1262.7,14052.641567,283.262188,5467.899590,5476,2.0,2.0,0
123,3586,62.927210,1,3709.8,8082.5,5973.9,13831.9,109396.4,29020.2,1464.9,1306.0,16415.510277,342.644554,6638.502535,6423,1.0,2.0,1


In [43]:
volumesDf.to_csv("../data/volume-data/preMatchVolumes.csv")

### Analyze stats before matching

In [4]:
def print_variable_stats(df):
    print(f"Progressive group - M/F gender: {df[(df['gen']==1) & (df['group']==1)].shape[0]}/{df[(df['gen']==2) & (df['group']==1)].shape[0]}")
    print(f"Stable group - M/F gender: {df[(df['gen']==1) & (df['group']==0)].shape[0]}/{df[(df['gen']==2) & (df['group']==0)].shape[0]}")

    age_prog_mean = df[df["group"]==1]["age"].mean()
    age_prog_std = df[df["group"]==1]["age"].std()
    age_stable_mean = df[df["group"]==0]["age"].mean()
    age_stable_std = df[df["group"]==0]["age"].std()
    print(f"Progressive group - Age: {age_prog_mean} +- {age_prog_std}")
    print(f"Stable group - Age: {age_stable_mean} +- {age_stable_std}")

    hy_stable_stage1_bl = len(df[df["initialHY"]==1])
    hy_stable_stage2_bl = len(df[df["initialHY"]==2])
    hy_stable_stage1_3y = len(df[df["followUpHY"]==1])
    hy_stable_stage2_3y = len(df[df["followUpHY"]==2])
    
    print(f"H&Y baseline stage 1: {hy_stable_stage1_bl}")
    print(f"H&Y baseline stage 2: {hy_stable_stage2_bl}")
    print(f"H&Y 3Y follow-up stage 1: {hy_stable_stage1_3y}")
    print(f"H&Y 3Y follow-up stage 2: {hy_stable_stage2_3y}")
    
    print(f"Stable dataset size: {len(df[df['group'] == 0])}")
    print(f"Progressive dataset size: {len(df[df['group'] == 1])}")
    
print_variable_stats(volumesDf)

Progressive group - M/F gender: 33/15
Stable group - M/F gender: 49/28
Progressive group - Age: 60.54356644710416 +- 10.246004545004064
Stable group - Age: 61.396429567870136 +- 9.26828414251294
H&Y baseline stage 1: 59
H&Y baseline stage 2: 66
H&Y 3Y follow-up stage 1: 23
H&Y 3Y follow-up stage 2: 94
Stable dataset size: 77
Progressive dataset size: 48


## Perform cohort matching

Now that we have our `volumesDf` ready, let's perform cohort matching on stable (0) and progressing (1) groups based on age, gender and initialHY. This will be done using the R script `match-data.R`

In [62]:
os.system("Rscript match-data.R")


Call:
matchit(formula = group ~ age * gen, data = data, method = "nearest", 
    distance = "glm", replacement = F)

Summary of Balance for All Data:
         Means Treated Means Control Std. Mean Diff. Var. Ratio eCDF Mean
distance        0.4240        0.3591          0.5701     0.8345    0.1391
age            60.5436       61.3964         -0.0832     1.2221    0.0533
gen             1.3125        1.3636         -0.1092     0.9359    0.0256
         eCDF Max
distance   0.2451
age        0.1285
gen        0.0511


Summary of Balance for Matched Data:
         Means Treated Means Control Std. Mean Diff. Var. Ratio eCDF Mean
distance        0.4240        0.4081          0.1396     1.5524    0.0337
age            60.5436       60.4474          0.0094     1.2231    0.0337
gen             1.3125        1.2708          0.0890     1.0879    0.0208
         eCDF Max Std. Pair Dist.
distance   0.1667          0.1514
age        0.1250          0.3763
gen        0.0417          0.4448

Percent B

0

In [8]:
# Import matched data
matchedVolumesDf = pd.read_csv("../data/volume-data/matchedVolumes.csv")
matchedVolumesDf = matchedVolumesDf.drop(["Unnamed: 0", "X", 'distance', 'weights', 'subclass'], 1)
matchedVolumesDf["subjectId"].values
# matchedVolumesDf[(matchedVolumesDf["group"]==0)]["initialHY"]

/Users/mohanadarafe/miniconda3/envs/torch/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  This is separate from the ipykernel package so we can avoid doing imports until


array([4037, 3168, 3131, 4024, 4001, 3373, 4081, 3127, 3107, 3134, 3825,
       3307, 3124, 3577, 3778, 3775, 3818, 4029, 3559, 4020, 3387, 4012,
       3111, 3835, 3366, 3831, 3591, 3826, 4027, 4135, 4026, 3371, 3323,
       3752, 3175, 3308, 3374, 3166, 3829, 3787, 3309, 3770, 3332, 3182,
       4033, 3174, 4083, 3173, 3176, 4034, 3113, 3125, 3377, 4005, 3102,
       3587, 4082, 3802, 3150, 3757, 3585, 3185, 3777, 3120, 3190, 3814,
       3305, 3808, 3311, 3364, 3378, 3832, 3588, 3383, 3866, 3116, 3828,
       3815, 3819, 3781, 3178, 3108, 3365, 3593, 3564, 4021, 3328, 3830,
       3758, 3823, 3325, 3132, 3385, 3822, 3372, 3586])